In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
conditions = [
    (train['Pawpularity'] >= 80),
    (train['Pawpularity'] < 80) & (train['Pawpularity'] >= 60),
    (train['Pawpularity'] < 60) & (train['Pawpularity'] >= 40),
    (train['Pawpularity'] < 40) & (train['Pawpularity'] >= 20),
    (train['Pawpularity'] < 20) & (train['Pawpularity'] >= 0),
]
values = ['tier_5','tier_4', 'tier_3', 'tier_2', 'tier_1']
train['tier'] = np.select(conditions, values)

In [ ]:
train = train.drop(columns='Id')

In [ ]:
from sklearn.model_selection import train_test_split
X = train[['Eyes','Face','Near','Group','Collage','Occlusion']]
y = train['Pawpularity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# from sklearn.neighbors import KNeighborsRegressor
# model = KNeighborsRegressor(n_neighbors=5)
# model.fit(X_train,y_train)
# model.score(X_test,y_test)

In [ ]:
# from sklearn.linear_model import LogisticRegressionCV
# model = LogisticRegressionCV(cv=3,solver='liblinear').fit(X_train, y_train)
# model.score(X_test,y_test)

In [ ]:
# from xgboost import XGBRegressor
# model =  XGBRegressor(
#         random_state=42,
#         n_estimators=10000,
# )
# model.fit(X_train,y_train)
# model.score(X_test,y_test)

In [ ]:
import lightgbm as lgb
model = lgb.LGBMRegressor(num_leaves=50,
                        learning_rate=0.1,
                        n_estimators=10000)
model.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l2',
        early_stopping_rounds=25)

In [ ]:
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
Test_X = test[['Eyes','Face','Near','Group','Collage','Occlusion']]
prediction = model.predict(Test_X)
prediction

In [ ]:
count = 0
data = []
for i in test.itertuples():
  data.append([i[1],int(prediction[count].round())])
  count+=1
data

In [ ]:
prediction_df = pd.DataFrame(data,columns=['Id','Pawpularity'])
prediction_df.to_csv('submission.csv',header=True,index=None)